## 머신러닝 실습

### 텍스트 마이닝

#### 개념

##### 텍스트 마이닝
- 비정형 텍스트데이터에서 패턴을 찾음, 의미가 있는 정보들을 추출하고 분석하는 기법
- 데이터 마이닝, 자연어 처리, 정보 검색 등의 분야가 결합된 분석기법
- 프로세스 - 텍스트전처리 > 특성 백터화 > 머신러닝 구축 학습, 학습, 평가 
- 텍스트 전처리 - 토큰화, 불용어 제거, 표제어 추출, 형태소 분석 ...

##### 특성 백터화/ 추출
- 컴퓨터는 글을 읽을 수 없음 -> 단어기반으로 특성 추출, 숫자형 백터값으로 표현
- Bow, Word2ve 라이브러리(모듗) 존재

##### LDA
 - 문서에 잠재되어 있는 토픽을 추론하는 확률 모델 알고리즘
 - pyLDAvis 시각화 라이브러리 사용

#### 데이터 수집
- https://github.com/e9t/nsmc
- ratings.txt, ratings_test.txt, ratings_train.txt

- id(리뷰번호), document(리뷰), label(감성분류클래스 0은 부정, 1은 긍정 감성)

In [1]:
## 필요 라이브러리 사용등록
import pandas as pd
import re

In [2]:
## 모든 모듈/라이브러리는 __version__
pd.__version__

'2.2.1'

In [3]:
## 버전 상세정보 , 판다스만 사용가능
pd.show_versions()


INSTALLED VERSIONS
------------------
commit                : bdc79c146c2e32f2cab629be240f01658cfb6cc2
python                : 3.12.2.final.0
python-bits           : 64
OS                    : Darwin
OS-release            : 23.3.0
Version               : Darwin Kernel Version 23.3.0: Wed Dec 20 21:30:59 PST 2023; root:xnu-10002.81.5~7/RELEASE_ARM64_T6030
machine               : arm64
processor             : arm
byteorder             : little
LC_ALL                : None
LANG                  : None
LOCALE                : None.UTF-8

pandas                : 2.2.1
numpy                 : 1.26.4
pytz                  : 2024.1
dateutil              : 2.9.0.post0
setuptools            : 69.1.1
pip                   : 24.0
Cython                : None
pytest                : None
hypothesis            : None
sphinx                : None
blosc                 : None
feather               : None
xlsxwriter            : None
lxml.etree            : 5.1.0
html5lib              : None
pymysql  

In [4]:
# 훈련용 데이터 가져오기
dfNsmcTrain = pd.read_csv('./data/ratings_train.txt', engine='python', sep='\t', encoding='utf-8')

In [5]:
dfNsmcTrain.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [6]:
dfNsmcTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [7]:
## 결측치 확인
dfNsmcTrain[dfNsmcTrain['document'].isnull()]

,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


In [8]:
dfNsmcTrain = dfNsmcTrain[dfNsmcTrain['document'].notnull()]

In [9]:
dfNsmcTrain.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [10]:
dfNsmcTrain['label'].value_counts()

label
0    75170
1    74825
Name: count, dtype: int64

In [11]:
## Regular Expression(정규식)으로 한글 이오의 문자는 다 공백으로 변경
dfNsmcTrain['document'] = dfNsmcTrain['document'].apply(lambda x: re.sub(r'[^가-힣:ㄱ-]+', ' ',x))

In [12]:
dfNsmcTrain[10:20]

,id,document,label
10,9008700,걍인피니트가짱이다 진짜짱이다,1
11,10217543,볼때마다 눈물나서 죽겠다 년대의 향수자극 허진호는 감성절제멜로의 달인이다,1
12,5957425,울면서 손들고 횡단보도 건널때 뛰쳐나올뻔 이범수 연기 드럽게못해,0
13,8628627,담백하고 깔끔해서 좋다 신문기사로만 보다 보면 자꾸 잊어버린다 그들도 사람이었다는 것을,1
14,9864035,취향은 존중한다지만 진짜 내생에 극장에서 본 영화중 가장 노잼 노감동임 스토리도 어...,0
15,6852435,ㄱ냥 매번 긴장되고 재밋음,1
16,9143163,참 사람들 웃긴게 바스코가 이기면 락스코라고 까고바비가 이기면 아이돌이라고 깐다 그...,1
17,4891476,굿바이 레닌 표절인것은 이해하는데 왜 뒤로 갈수록 재미없어지냐,0
18,7465483,이건 정말 깨알 캐스팅과 질퍽하지않은 산뜻한 내용구성이 잘 버무러진 깨알일드,1
19,3989148,약탈자를 위한 변명 이라 저놈들은 착한놈들 절대 아닌걸요,1


In [13]:
## 평가용 데이터 준비
dfNsmcTest = pd.read_csv('./data/ratings_test.txt', engine='python', sep= '\t', encoding='utf-8')

In [14]:
dfNsmcTest.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [15]:
dfNsmcTest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [16]:
dfNsmcTest[dfNsmcTest['document'].isnull()]

,id,document,label
5746,402110,NaN,1
7899,5026896,NaN,0
27097,511097,NaN,1


In [17]:
dfNsmcTest = dfNsmcTest[dfNsmcTest['document'].notnull()]

In [18]:
dfNsmcTest['label'].value_counts()

label
1    25171
0    24826
Name: count, dtype: int64

In [19]:
## Regular Expression(정규식) 으로 한글 이외의 문제는 다 공백으로 변경
dfNsmcTest['document'] = dfNsmcTest['document'].apply(lambda x: re.sub(r'[^ 가-힣|ㄱ-]+', '', x))

In [20]:
dfNsmcTest.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


In [21]:
## 한글 이외의 텍스트 삭제 후 document가 비어있는 ' ' 데이터를 제거
dfNsmcTrain = dfNsmcTrain[  dfNsmcTrain['document'] !='']

In [22]:
dfNsmcTrain.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [23]:
## 한글 이외의 텍스트 삭제 후 document가 비어있는 ' ' 데이터rk 554개 존재
dfNsmcTest = dfNsmcTest[  dfNsmcTest['document'] !='']

In [24]:
dfNsmcTest.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49598 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49598 non-null  int64 
 1   document  49598 non-null  object
 2   label     49598 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


##### 분석모델 구축

- 이전, 특징 벡터화 작업, 단어로 분리(토큰화) 후 TF-IDF 방식으로 벡터화
- 한들 형태소 분석은 konlpy 패키의 Okt 클래스 사용

##### KoBLPy 라이브러리 설치
- JDK 설치
- 시스템등록정보 JAVA_HOME 설정
    - 프로그램 실행: sysdm.cpl
    - 시스템속성 > 고급탭 > 환경변수 버튼 클릭
    - 시스템변수 > 새로만들기 버튼 클릭
    - JAVA_HOME 과 경로 추가 등록
- JPype1  - 파이썬에서 JDK를 사용하게 해주는 프로그램
- koNLPy 설치

In [25]:
# JPype1 설치
!pip3 install JPype1

In [26]:
# koNLPy 설치
!pip3 install koNLPy

In [27]:
import konlpy

In [28]:
# 특성 백터화 작업
from konlpy.tag import Okt

In [30]:
okt = Okt()

JVMNotFoundException: No JVM shared library file (libjli.dylib) found. Try setting up the JAVA_HOME environment variable properly.